In [9]:
# Importar a biblioteca requests
import requests
import pandas as pd
import seaborn as sns

In [2]:
# Definir a URL base da API
base_url = "https://dadosabertos.camara.leg.br/api/v2"

# Definir o id da legislatura atual
id_legislatura = 56

# Definir o ano desejado
ano = 2023

In [3]:
# Criar uma lista vazia para armazenar os dados
dados = []

In [4]:
# Fazer uma requisição para obter a lista de deputados da legislatura atual
response = requests.get(base_url + "/deputados", params={"idLegislatura": id_legislatura})

# Verificar se a requisição foi bem sucedida
if response.status_code == 200:
    # Obter os dados em formato JSON
    deputados = response.json()["dados"]

    # Para cada deputado na lista
    for deputado in deputados:
        # Obter o id, o nome, o partido e o estado do deputado
        id_deputado = deputado["id"]
        nome_deputado = deputado["nome"]
        partido_deputado = deputado["siglaPartido"]
        estado_deputado = deputado["siglaUf"]

        # Fazer uma requisição para obter as despesas do deputado no ano desejado
        response = requests.get(base_url + f"/deputados/{id_deputado}/despesas", params={"ano": ano})

        # Verificar se a requisição foi bem sucedida
        if response.status_code == 200:
            # Obter os dados em formato JSON
            despesas = response.json()["dados"]

            # Para cada despesa na lista
            for despesa in despesas:
                # Adicionar o id, o nome, o partido e o estado do deputado à despesa
                despesa["idDeputado"] = id_deputado
                despesa["nomeDeputado"] = nome_deputado
                despesa["partidoDeputado"] = partido_deputado
                despesa["estadoDeputado"] = estado_deputado

            # Adicionar os dados à lista
            dados.extend(despesas)


In [27]:
# Criar um data frame a partir da lista de dados
despesas = pd.DataFrame(dados)

In [5]:
# Salvar o data frame em um arquivo csv
despesas.to_csv("despesas_deputados.csv", index=False)

In [28]:
despesas.head()

,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,...,cnpjCpfFornecedor,valorLiquido,valorGlosa,numRessarcimento,codLote,parcela,idDeputado,nomeDeputado,partidoDeputado,estadoDeputado
0,2023,1,COMBUSTÍVEIS E LUBRIFICANTES.,7486858,Nota Fiscal Eletrônica,4,2023-01-15,409598,310.04,http://www.camara.leg.br/cota-parlamentar/nota...,...,13445527000171,310.04,0.0,,1902411,0,204554,Abílio Santana,PSC,BA
1,2023,1,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.",7487268,Nota Fiscal,0,2023-01-19,00000072,6000.00,https://www.camara.leg.br/cota-parlamentar/doc...,...,33903780000139,6000.00,0.0,,1902606,0,204554,Abílio Santana,PSC,BA
2,2023,1,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,7486370,Nota Fiscal,0,2023-01-17,00000061,6000.00,https://www.camara.leg.br/cota-parlamentar/doc...,...,32348287000131,6000.00,0.0,,1902201,0,204554,Abílio Santana,PSC,BA
3,2023,1,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,7485881,Nota Fiscal,0,2023-01-16,00000014,6000.00,https://www.camara.leg.br/cota-parlamentar/doc...,...,44255986000116,6000.00,0.0,,1901936,0,204554,Abílio Santana,PSC,BA
4,2023,1,SERVIÇOS POSTAIS,0,Recibos/Outros,1,2023-01-10,RSP : 2335618,318.48,None,...,00000000000007,318.48,0.0,0,0,0,204554,Abílio Santana,PSC,BA


In [34]:
despesas.nomeDeputado

0       Abílio Santana
1       Abílio Santana
2       Abílio Santana
3       Abílio Santana
4       Abílio Santana
             ...      
7297       Zeca Dirceu
7298       Zeca Dirceu
7299       Zeca Dirceu
7300       Zeca Dirceu
7301       Zeca Dirceu
Name: nomeDeputado, Length: 7302, dtype: object

In [29]:
despesas.shape

(7302, 21)

In [35]:
# Criar uma coluna com o mês da despesa
despesas["mes"] = pd.to_datetime(despesas["dataDocumento"]).dt.month

# Agrupar os dados por deputado, mês e tipo de despesa
gastos_por_tipo = despesas.groupby(['mes', 'ano', "tipoDespesa", "idDeputado", 
                                    'nomeDeputado', 'partidoDeputado', 'estadoDeputado'])["valorLiquido"].sum().reset_index()

# Agrupar os dados por deputado e mês
gastos_por_mes = despesas.groupby(["mes", 'ano', "idDeputado", 
                                    'nomeDeputado', 'partidoDeputado', 'estadoDeputado'])["valorLiquido"].sum().reset_index()

# Agrupar os dados por deputado
gastos_totais = despesas.groupby(["idDeputado", 'nomeDeputado', 
                                 'partidoDeputado', 'estadoDeputado'])["valorLiquido"].sum().reset_index()

# Renomear as colunas dos data frames
gastos_por_tipo = gastos_por_tipo.rename(columns={"valorLiquido": "gastoPorTipo"})
gastos_por_mes = gastos_por_mes.rename(columns={"valorLiquido": "gastoPorMes"})
gastos_totais = gastos_totais.rename(columns={"valorLiquido": "gastoTotal"})

In [44]:
# Juntar os data frames em um só
despesa_total = pd.merge(gastos_por_tipo, gastos_por_mes, on=["idDeputado", "mes", "ano"])
despesa_total = pd.merge(despesa_total, gastos_totais, on="idDeputado")

In [ ]:
despesa_total.drop(['nomeDeputado_y', 'partidoDeputado_y', 'estadoDeputado_y', 'nomeDeputado', 
                   'partidoDeputado', 'estadoDeputado'], axis=1, inplace=True)

In [50]:
despesa_total.rename(columns={"nomeDeputado_x": "nomeDeputado",
                                              "partidoDeputado_x": "partidoDeputado",
                                              "estadoDeputado_x": "estadoDeputado"}, inplace=True)

In [51]:
despesa_total.head(30)

,mes,ano,tipoDespesa,idDeputado,nomeDeputado,partidoDeputado,estadoDeputado,gastoPorTipo,gastoPorMes,gastoTotal
0,1.0,2023,ASSINATURA DE PUBLICAÇÕES,156190,Marcel van Hattem,NOVO,RS,634.80,8199.01,8203.11
1,1.0,2023,COMBUSTÍVEIS E LUBRIFICANTES.,156190,Marcel van Hattem,NOVO,RS,451.30,8199.01,8203.11
2,1.0,2023,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.",156190,Marcel van Hattem,NOVO,RS,1500.00,8199.01,8203.11
3,1.0,2023,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,156190,Marcel van Hattem,NOVO,RS,111.17,8199.01,8203.11
4,1.0,2023,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,156190,Marcel van Hattem,NOVO,RS,328.88,8199.01,8203.11
5,1.0,2023,PASSAGEM AÉREA - REEMBOLSO,156190,Marcel van Hattem,NOVO,RS,1039.73,8199.01,8203.11
6,1.0,2023,PASSAGEM AÉREA - SIGEPA,156190,Marcel van Hattem,NOVO,RS,4133.13,8199.01,8203.11
7,1.0,2023,ASSINATURA DE PUBLICAÇÕES,204440,Christino Aureo,PP,RJ,134.90,31370.87,31426.59
8,1.0,2023,COMBUSTÍVEIS E LUBRIFICANTES.,204440,Christino Aureo,PP,RJ,50.00,31370.87,31426.59
9,1.0,2023,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,204440,Christino Aureo,PP,RJ,14300.00,31370.87,31426.59


In [86]:
despesa_total_crescente = despesa_total.sort_values(by="gastoTotal")
despesa_total_crescente

,mes,ano,tipoDespesa,idDeputado,nomeDeputado,partidoDeputado,estadoDeputado,gastoPorTipo,gastoPorMes,gastoTotal
2195,1.0,2023,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,74471,André de Paula,PSD,PE,119.70,119.70,122.55
2408,1.0,2023,SERVIÇOS POSTAIS,74399,Onyx Lorenzoni,PL,RS,400.27,518.17,636.07
2409,1.0,2023,TELEFONIA,74399,Onyx Lorenzoni,PL,RS,117.90,518.17,636.07
2410,2.0,2023,TELEFONIA,74399,Onyx Lorenzoni,PL,RS,117.90,117.90,636.07
2419,1.0,2023,TELEFONIA,178922,Tadeu Alencar,PSB,PE,707.40,707.40,1061.10
...,...,...,...,...,...,...,...,...,...,...
1885,1.0,2023,PASSAGEM AÉREA - SIGEPA,74356,Silas Câmara,REPUBLICANOS,AM,7572.79,60090.22,160683.75
1889,4.0,2023,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,74356,Silas Câmara,REPUBLICANOS,AM,30000.00,30000.00,160683.75
1888,3.0,2023,LOCAÇÃO OU FRETAMENTO DE AERONAVES,74356,Silas Câmara,REPUBLICANOS,AM,23000.00,68600.00,160683.75
1887,3.0,2023,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,74356,Silas Câmara,REPUBLICANOS,AM,45600.00,68600.00,160683.75


In [88]:
# Remover linhas duplicadas mantendo apenas a primeira ocorrência para cada combinação de mês, ano e nomeDeputado
despesa_total_crescente_limpo = despesa_total_crescente.drop_duplicates(subset=['ano', 'nomeDeputado'], keep='first')
despesa_total_crescente_limpo

,mes,ano,tipoDespesa,idDeputado,nomeDeputado,partidoDeputado,estadoDeputado,gastoPorTipo,gastoPorMes,gastoTotal
2195,1.0,2023,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,74471,André de Paula,PSD,PE,119.70,119.70,122.55
2408,1.0,2023,SERVIÇOS POSTAIS,74399,Onyx Lorenzoni,PL,RS,400.27,518.17,636.07
2419,1.0,2023,TELEFONIA,178922,Tadeu Alencar,PSB,PE,707.40,707.40,1061.10
944,1.0,2023,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO ...",178935,Rossoni,PSDB,PR,150.00,1509.71,1510.52
2286,1.0,2023,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,179587,Major Vitor Hugo,PL,GO,1684.39,1684.39,1684.39
...,...,...,...,...,...,...,...,...,...,...
1255,2.0,2023,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,204390,Professor Alcides,PL,GO,15000.00,35000.00,101117.81
2480,4.0,2023,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,123756,Antônia Lúcia,REPUBLICANOS,AC,5600.00,67600.00,108027.98
2298,3.0,2023,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,204419,Glaustin da Fokus,PSC,GO,78.73,89087.73,125691.58
1999,1.0,2023,PASSAGEM AÉREA - SIGEPA,74383,Giacobo,PL,PR,5527.43,24939.11,142994.91
